In [1]:
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd

## Search filters

In [75]:
position = "Data Engineer"
location = "France"

## Settings headers

In [7]:
# Define a list of different headers to mimic various browser and OS combinations
headers = [
    {'User-Agent': 'Mozilla/5.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36'}
]

# Randomly choose one of the headers to use for the request
header = random.choice(headers)
print(f"Using header: {header}")

Using header: {'User-Agent': 'Mozilla/5.0'}


## Fetch list of jobPosting (ids)

- https://blog.devgenius.io/web-scraping-linkedin-job-postings-e93d14b6b68f
- https://blog.devgenius.io/web-scraping-linkedin-job-postings-part-2-caddc463d324
- https://blog.devgenius.io/web-scraping-linkedin-job-postings-part-3-data-cleaning-and-categorization-cd87532ef046

In [8]:
# url = f"https://www.linkedin.com/jobs/search?keywords={position}&location={location}&position=1&pageNum=0"
url = f"https://www.linkedin.com/jobs/search?keywords={position}&location={location}"

In [9]:
# list_url = f"https://www.linkedin.com/jobs/search?keywords=Data%20Engineer&location=France&position=1&pageNum=0"
# https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Engineer&location=France&currentJobId=4302070575&start=0&f_WT=2

In [10]:
# https://www.linkedin.com/jobs/search?keywords=Data%20Engineer&location=France&position=1&pageNum=0
# https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Engineer&location=France&distance=20&f_TPR=r86400&start=0&f_WT=2
# https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Engineer&location=France&distance=20&f_TPR=r86400

In [11]:
# https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Engineer&location=France&currentJobId=4302070575&start=0&f_WT=2
# https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4302070575'

In [12]:
response = requests.get(url, headers=header)

In [16]:
# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [59]:
# Find the unordered list (ul) element that contains the job search results using its class name
soup_jobs = soup.find("ul", class_="jobs-search__results-list")

# Find all list items (li) within the joblist, representing individual job postings
soup_jobs_li = soup_jobs.find_all('li')

In [60]:
soup_jobs_li[0]

<li>
<div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card" data-column="1" data-entity-urn="urn:li:jobPosting:4295805013" data-impression-id="jobs-search-desktop-0" data-reference-id="mrydzisbQA+Dt7ipFjmOug==" data-row="1" data-tracking-id="CBBMoySqGLvwFtyq4XYdnA==">
<a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2] outline-offset-[4px]" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://fr.linkedin.com/jobs/view/junior-data-engineer-f-m-at-vinci-airports-4295805013?position=1&amp;pageNum=0&amp;refId=mrydzisbQA%2BDt7ipFjmOug%3D%3D&amp;trackingId=CBBMoySqGLvwFtyq4XYdnA%3D%3D">
<span class="sr-only">
              
        
        Junior Data Engineer (F/M)
      
      
          </span>
</a>
<div class="search-entity-media">
<img alt="" class="artdeco-entity-image artdeco

In [68]:
jobs = []
for job in soup_jobs_li:
    
    posting_id = job.find("div", class_="base-card").get("data-entity-urn").split(":")[3] if job.find("div", class_="base-card") else None
    title = job.find("h3", class_="base-search-card__title").text.strip() if job.find("h3", class_="base-search-card__title") else None
    company = job.find("h4", class_="base-search-card__subtitle").text.strip() if job.find("h4", class_="base-search-card__subtitle") else None
    location = job.find("span", class_="job-search-card__location").text.strip() if job.find("span", class_="job-search-card__location") else None
    url = job.find("a", class_="base-card__full-link").get("href") if job.find("a", class_="base-card__full-link") else None

    job_details = {
        "id" : posting_id,
        "title" : title,
        "company" : company,
        "location" : location,
        "url": url        
    }

    jobs.append(job_details)

In [69]:
len(jobs)

60

In [70]:
jobs[0]

{'id': '4295805013',
 'title': 'Junior Data Engineer (F/M)',
 'company': 'VINCI Airports',
 'location': 'Nanterre, Île-de-France, France',
 'url': 'https://fr.linkedin.com/jobs/view/junior-data-engineer-f-m-at-vinci-airports-4295805013?position=1&pageNum=0&refId=mrydzisbQA%2BDt7ipFjmOug%3D%3D&trackingId=CBBMoySqGLvwFtyq4XYdnA%3D%3D'}

In [80]:
# Dictionary to map time posted options to LinkedIn's URL parameter values
time_posted_dict = {
    'ALL': '',
    'MONTH': 'r2592000',
    'WEEK': 'r604800',
    'DAY': 'r86400' # r86400 refers to the amount of seconds passed, in this case 24*60*60 = 86400
}

# Dictionary to map remote work options to LinkedIn's URL parameter values
remote_dict = {
    'ALL': '',
    'ON-SITE': '1',
    'REMOTE': '2',
    'HYBRID': '3'
}

In [82]:
def generate_linkedin_url(position: str, location: str, distance: int = 10, 
                          time_posted: str = "ALL", remote: str = "ALL") -> str:
    
    # Base URL for Linkedin job search1
    base_url = "https://www.linkedin.com/jobs/search"

    # The keywords and location are required parameters.
    query_params = f"?keywords={position.replace(' ', '%20')}&location={location.replace(' ', '%20')}&distance={distance}"

    # Adding time to query params
    query_params += f"&f_TPR={time_posted_dict.get(time_posted, '')}"

    # Adding remote to query params
    query_params += f"&f_WT={remote_dict.get(remote, '')}"

    return base_url + query_params

    

main_url = generate_linkedin_url(
    position="Data Scientist", 
    location="Monterrey", 
    distance=10, 
    time_posted="WEEK", 
    remote="REMOTE"
)

print(main_url)
    

https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location=Monterrey&distance=10&f_TPR=r604800&f_WT=2


In [57]:
job_id = "4291140251"
job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
job_response = requests.get(job_url)

In [58]:
print(BeautifulSoup(job_response.text, "html.parser"))


<!-- -->
<section class="top-card-layout container-lined overflow-hidden babybear:rounded-[0px]">
<div class="top-card-layout__card relative p-2 papabear:p-details-container-padding">
<a data-tracking-control-name="public_jobs_topcard_logo" data-tracking-will-navigate="" href="https://fr.linkedin.com/company/orange-business?trk=public_jobs_topcard_logo" target="_self">
<img alt="Orange Business" class="artdeco-entity-image artdeco-entity-image--square-5" data-delayed-url="https://media.licdn.com/dms/image/v2/C4E0BAQHkuI2zyi1F8Q/company-logo_100_100/company-logo_100_100/0/1676538388568/orange_business_services_logo?e=2147483647&amp;v=beta&amp;t=a9t4M6vyaeecEQ2a2o5epemkc-6QWfV-YNROeDm-iZc" data-ghost-classes="artdeco-entity-image--ghost" data-ghost-url="https://static.licdn.com/aero-v1/sc/h/aajlclc14rr2scznz5qm2rj9u"/>
</a>
<div class="top-card-layout__entity-info-container flex flex-wrap papabear:flex-nowrap">
<div class="top-card-layout__entity-info flex-grow flex-shrink-0 basis-0 bab

In [84]:
from urllib.parse import urlencode, quote_plus

base_url = "https://www.linkedin.com/jobs/search"

params = {
        "keywords": "Data Engineer",
        "location": "France",
        "distance": 10
    }

print(f"{base_url}?{urlencode(params, quote_via=quote_plus)}")

https://www.linkedin.com/jobs/search?keywords=Data+Engineer&location=France&distance=10
